Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/regression-part2-automated-ml.png)

# Tutorial: Use automated machine learning to predict taxi fares

In this tutorial, you use automated machine learning in Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this tutorial you learn the following tasks:

* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you donâ€™t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Complete the [setup tutorial](https://docs.microsoft.com/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) if you don't already have an Azure Machine Learning service workspace or notebook virtual machine.
* After you complete the setup tutorial, open the **tutorials/regression-automated-ml.ipynb** notebook using the same notebook server.

This tutorial is also available on [GitHub](https://github.com/Azure/MachineLearningNotebooks/tree/master/tutorials) if you wish to run it in your own [local environment](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/README.md#setup-using-a-local-conda-environment).

## Download and prepare data

In [1]:
import pandas as pd
from azureml.core import Dataset
from datetime import datetime
from dateutil.relativedelta import relativedelta

Begin by creating a dataframe to hold the taxi data. Then preview the data.

In [2]:
green_taxi_dataset = Dataset.Tabular.from_parquet_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/green_taxi_data.parquet")
green_taxi_df = green_taxi_dataset.to_pandas_dataframe()
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,__index_level_0__
0,2,2015-01-30 18:38:09,2015-01-30 19:01:49,1,1.88,None,None,-73.996155,40.690903,-73.964287,...,15.0,1.0,0.5,0.3,4.00,0.0,None,20.80,1.0,2015-01-30 18:38:09
1,1,2015-01-17 23:21:39,2015-01-17 23:35:16,1,2.70,None,None,-73.978508,40.687984,-73.955116,...,11.5,0.5,0.5,0.3,2.55,0.0,None,15.35,1.0,2015-01-17 23:21:39
2,2,2015-01-16 01:38:40,2015-01-16 01:52:55,1,3.54,None,None,-73.957787,40.721779,-73.963005,...,13.5,0.5,0.5,0.3,2.80,0.0,None,17.60,1.0,2015-01-16 01:38:40
3,2,2015-01-04 17:09:26,2015-01-04 17:16:12,1,1.00,None,None,-73.919914,40.826023,-73.904839,...,6.5,0.0,0.5,0.3,0.00,0.0,None,7.30,1.0,2015-01-04 17:09:26
4,1,2015-01-14 10:10:57,2015-01-14 10:33:30,1,5.10,None,None,-73.943710,40.825439,-73.982964,...,18.5,0.0,0.5,0.3,3.85,0.0,None,23.15,1.0,2015-01-14 10:10:57
5,2,2015-01-19 18:10:41,2015-01-19 18:32:20,1,7.41,None,None,-73.940918,40.839714,-73.994339,...,24.0,0.0,0.5,0.3,4.80,0.0,None,29.60,1.0,2015-01-19 18:10:41
6,2,2015-01-01 15:44:21,2015-01-01 15:50:16,1,1.03,None,None,-73.985718,40.685646,-73.996773,...,6.5,0.0,0.5,0.3,1.30,0.0,None,8.60,1.0,2015-01-01 15:44:21
7,2,2015-01-12 08:01:21,2015-01-12 08:14:52,5,2.94,None,None,-73.939865,40.789822,-73.952957,...,12.5,0.0,0.5,0.3,0.00,0.0,None,13.30,1.0,2015-01-12 08:01:21
8,1,2015-01-16 21:54:26,2015-01-16 22:12:39,1,3.00,None,None,-73.957939,40.721928,-73.926247,...,14.0,0.5,0.5,0.3,2.00,0.0,None,17.30,1.0,2015-01-16 21:54:26
9,2,2015-01-06 06:34:53,2015-01-06 06:44:23,1,2.31,None,None,-73.943825,40.810257,-73.943062,...,10.0,0.0,0.5,0.3,2.00,0.0,None,12.80,1.0,2015-01-06 06:34:53


Remove some of the columns that you won't need for training or additional feature building.  Automate machine learning will automatically handle time-based features such as lpepPickupDatetime.

In [3]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,__index_level_0__
0,2,2015-01-30 18:38:09,1,1.88,-73.996155,40.690903,-73.964287,40.679707,20.80,2015-01-30 18:38:09
1,1,2015-01-17 23:21:39,1,2.70,-73.978508,40.687984,-73.955116,40.708138,15.35,2015-01-17 23:21:39
2,2,2015-01-16 01:38:40,1,3.54,-73.957787,40.721779,-73.963005,40.682774,17.60,2015-01-16 01:38:40
3,2,2015-01-04 17:09:26,1,1.00,-73.919914,40.826023,-73.904839,40.821404,7.30,2015-01-04 17:09:26
4,1,2015-01-14 10:10:57,1,5.10,-73.943710,40.825439,-73.982964,40.767857,23.15,2015-01-14 10:10:57


### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [4]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.777625,1.373625,2.893981,-73.827403,40.689730,-73.819670,40.684436,14.892744
std,0.415850,1.046180,3.072343,2.821767,1.556082,2.901199,1.599776,12.339749
min,1.000000,0.000000,0.000000,-74.357101,0.000000,-74.342766,0.000000,-120.800000
25%,2.000000,1.000000,1.050000,-73.959175,40.699127,-73.966476,40.699459,8.000000
50%,2.000000,1.000000,1.930000,-73.945049,40.746754,-73.944221,40.747536,11.300000
75%,2.000000,1.000000,3.700000,-73.917089,40.803060,-73.909061,40.791526,17.800000
max,2.000000,8.000000,154.280000,0.000000,41.109089,0.000000,40.982826,425.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [5]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [6]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount
count,23222.000000,23222.000000,23222.000000,23222.000000
mean,1.778572,1.374688,2.956753,14.838994
std,0.415217,1.046995,2.862415,10.363600
min,1.000000,1.000000,0.250000,0.010000
25%,2.000000,1.000000,1.100000,8.190000
50%,2.000000,1.000000,2.000000,11.750000
75%,2.000000,1.000000,3.760000,17.880000
max,2.000000,8.000000,30.840000,191.700000


## Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [7]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RRS23QZ5P to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
72f988bf-86f1-41af-91ab-2d7cd011db47 'Microsoft'
The following tenants require Multi-Factor Authentication (MFA). Use 'az login --tenant TENANT_ID' to explicitly login to a tenant.
76641144-f9ac-402f-90c1-d7eb04361ba0 'Aelterman.Info B2C'


Interactive authentication successfully completed.


## Split the data into train and test sets

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(final_df, test_size=0.2, random_state=223)

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 20 minutes, but if you want a shorter run time, reduce the `experiment_timeout_hours` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|10|Time limit in minutes for each iteration. Increase this value for larger datasets that need more time for each iteration.|
|**experiment_timeout_hours**|0.3|Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**|True|Flag to enable early termination if the score is not improving in the short term.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**featurization**| auto | By using auto, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|

In [9]:
import logging

automl_settings = {
	"iterations": 5,
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'spearman_correlation',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [10]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             label_column_name="totalAmount",
                             **automl_settings)

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

In [11]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "Tutorial-NYCTaxi")
local_run = experiment.submit(automl_config, show_output=True)

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /opt/conda/lib/python3.9/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
azure-mgmt-resource<=21.1.0/azure-mgmt-resource 21.2.0
bcrypt<=4.0.0/bcrypt 4.0.1
distro<=1.7.0/distro 1.8.0
jmespath<=0.10.0/jmespath 1.0.1
numpy<=1.22.3/numpy 1.23.4
oauthlib<=3.2.1/oauthlib 3.2.2
pandas<=1.1.5/pandas 1.5.1
portalocker<=2.5.1/portalocker 2.6.0
pytz<=2022.4/pytz 2022.5
types-cryptography<=3.3.23/types-cryptography 3.3.23.1
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /opt/conda/lib/python3.9/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, u

## Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.

In [ ]:
y_test = x_test.pop("totalAmount")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Compute**.
1. Select the **Notebook VMs** tab in the compute page.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this automated machine learning tutorial, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using an automated regression model locally with custom parameters.
> * Explored and reviewed training results.

[Deploy your model](https://docs.microsoft.com/azure/machine-learning/service/tutorial-deploy-models-with-aml) with Azure Machine Learning service.